In [1]:
%%capture
!pip install gdown contextily

In [2]:
import gdown

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import contextily as ctx
import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Embedding, Concatenate, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import feature_column as fc

from sklearn.model_selection import train_test_split
from geopy.distance import geodesic

In [3]:
%%capture

ROUTE_GPX = 'route.gpx'
ROUTE_GPX_URL = 'https://drive.google.com/file/d/1-kx84-fNAOuDWSdVEYU_DjP4Om0yHnaD/view?usp=sharing'
gdown.download(ROUTE_GPX_URL, ROUTE_GPX, quiet=True, fuzzy=True)

In [4]:
def parse_gpx(filepath):
    root = ET.parse(filepath).getroot()

    ns = '{http://www.topografix.com/GPX/1/1}'
    data = {
        'time': [], 'lat': [], 'lon': [],
    }

    for trk in root.findall(f".//{ns}trk"):
        for trkseg in trk.findall(f"{ns}trkseg"):
            for trkpt in trkseg.findall(f"{ns}trkpt"):
                data['lat'].append(float(trkpt.get('lat')))
                data['lon'].append(float(trkpt.get('lon')))

                time_elem = trkpt.find(f"{ns}time")
                data['time'].append(time_elem.text if time_elem is not None else None)

    df = pd.DataFrame(data)
    return df

df = parse_gpx(ROUTE_GPX)

df['time'] = pd.to_datetime(df['time'])
df['time'] = df['time'].dt.tz_convert('UTC')
df['time'] = (df['time'] - pd.Timestamp("1970-01-01", tz='UTC')) // pd.Timedelta('1s')

df.head()

,time,lat,lon
0,1730128585,20.993817,105.868370
1,1730128586,20.993691,105.868892
2,1730128587,20.993616,105.869054
3,1730128588,20.993609,105.869099
4,1730128589,20.993607,105.869111


In [56]:
def get_bounding_box(df):
    min_lat = df['lat'].min()
    max_lat = df['lat'].max()
    min_lon = df['lon'].min()
    max_lon = df['lon'].max()
    bottom_left = (min_lat, min_lon)
    top_right = (max_lat, max_lon)
    return (bottom_left, top_right)

bottom_left, top_right = get_bounding_box(df)
print(bottom_left, top_right)

(20.990503099197838, 105.86562686937982) (20.9968994182943, 105.8707035101781)


In [57]:
# Define how many buckets you want for each dimension
num_lat_buckets = 100
num_lon_buckets = 100

bottom_left_lat, bottom_left_lon = bottom_left
top_right_lat, top_right_lon = top_right

lat_range = top_right_lat - bottom_left_lat
lon_range = top_right_lon - bottom_left_lon

boundaries_latitude = [bottom_left_lat + (i * (lat_range / num_lat_buckets)) for i in range(1, num_lat_buckets)]
boundaries_longitude = [bottom_left_lon + (i * (lon_range / num_lon_buckets)) for i in range(1, num_lon_buckets)]

print(boundaries_latitude)
print(boundaries_longitude)

[20.990567062388802, 20.990631025579766, 20.99069498877073, 20.990758951961695, 20.990822915152663, 20.990886878343627, 20.99095084153459, 20.991014804725555, 20.99107876791652, 20.991142731107484, 20.991206694298448, 20.991270657489412, 20.991334620680377, 20.991398583871344, 20.99146254706231, 20.991526510253273, 20.991590473444237, 20.9916544366352, 20.991718399826166, 20.99178236301713, 20.991846326208094, 20.99191028939906, 20.991974252590026, 20.99203821578099, 20.992102178971955, 20.99216614216292, 20.992230105353883, 20.992294068544847, 20.99235803173581, 20.992421994926776, 20.99248595811774, 20.992549921308708, 20.992613884499672, 20.992677847690636, 20.9927418108816, 20.992805774072565, 20.99286973726353, 20.992933700454493, 20.992997663645458, 20.993061626836422, 20.99312559002739, 20.993189553218354, 20.993253516409318, 20.993317479600282, 20.993381442791247, 20.99344540598221, 20.993509369173175, 20.99357333236414, 20.993637295555104, 20.99370125874607, 20.993765221937036

In [64]:
# Define time input
time_input = Input(shape=(1,), name="time")
time_normalized = tf.keras.layers.Normalization()(time_input)

# Define latitude and longitude inputs
latitude_input = Input(shape=(1,), name="latitude")
longitude_input = Input(shape=(1,), name="longitude")

# Apply normalization and discretization (bucketizing)
latitude_normalized = tf.keras.layers.Normalization()(latitude_input)
longitude_normalized = tf.keras.layers.Normalization()(longitude_input)

latitude_bucketized = tf.keras.layers.Discretization(bin_boundaries=boundaries_latitude)(latitude_normalized)
longitude_bucketized = tf.keras.layers.Discretization(bin_boundaries=boundaries_longitude)(longitude_normalized)

# Concatenate the inputs
concatenated_inputs = tf.keras.layers.Concatenate()([time_normalized, latitude_bucketized, longitude_bucketized])

# Create a model using the Functional API
x = Dense(128, activation='relu')(concatenated_inputs)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)

# Output layer for predicting latitude and longitude
output = Dense(2, activation='linear', name='predicted_coordinates')(x)

def scale_output(x):
    min_lat, min_lon = bottom_left
    max_lat, max_lon = top_right
    lat = (x[:, 0] + 1) / 2 * (max_lat - min_lat) + min_lat  # Scaling from [-1, 1] to [min_lat, max_lat]
    lon = (x[:, 1] + 1) / 2 * (max_lon - min_lon) + min_lon  # Scaling from [-1, 1] to [min_lon, max_lon]
    return tf.stack([lat, lon], axis=1)

# Apply custom scaling layer
scaled_output = tf.keras.layers.Lambda(scale_output)(output)

# Define the model
model = Model(inputs=[time_input, latitude_input, longitude_input], outputs=scaled_output)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Print the model summary
model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ latitude (InputLayer)     │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ longitude (InputLayer)    │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time (InputLayer)         │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_51          │ (None, 1)              │              3 │ latitude[0][0]         │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_52          │ (None, 1)              │              3 │ longitude[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_50          │ (None, 1)              │              3 │ time[0][0]             │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ discretization_36         │ (None, 1)              │              0 │ normalization_51[0][0] │
│ (Discretization)          │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ discretization_37         │ (None, 1)              │              0 │ normalization_52[0][0] │
│ (Discretization)          │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_11            │ (None, 3)              │              0 │ normalization_50[0][0… │
│ (Concatenate)             │                        │                │ discretization_36[0][… │
│                           │                        │                │ discretization_37[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_44 (Dense)          │ (None, 128)            │            512 │ concatenate_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_21 (Dropout)      │ (None, 128)            │              0 │ dense_44[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_45 (Dense)          │ (None, 64)             │          8,256 │ dropout_21[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_22 (Dropout)      │ (None, 64)             │              0 │ dense_45[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_46 (Dense)          │ (None, 32)             │          2,080 │ dropout_22[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_23 (Dropout)      │ (None, 32)             │              0 │ dense_46[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ predicted_coordinates     │ (None, 2)              │             66 │ dropout_23[0][0]       │
│ (Dense)              

 Total params: 10,923 (42.68 KB)

 Trainable params: 10,914 (42.63 KB)

 Non-trainable params: 9 (48.00 B)

In [65]:
X = df[['time', 'lat', 'lon']]
y = df[['lat', 'lon']]

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_split = {
    'latitude': X_train['lat'].values,
    'longitude': X_train['lon'].values,
    'time': X_train['time'].values,
}
X_val_split = {
    'latitude': X_val['lat'].values,
    'longitude': X_val['lon'].values,
    'time': X_val['time'].values,
}

# Ensure y_train and y_val are structured correctly
y_train_combined = np.column_stack((y_train['lat'].values, y_train['lon'].values))
y_val_combined = np.column_stack((y_val['lat'].values, y_val['lon'].values))

# Fit the model
model.fit(
    x=X_train_split,
    y=y_train_combined,  # Use combined array
    validation_data=(X_val_split, y_val_combined),  # Use combined array
    epochs=100,
    batch_size=32,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)]
)

Epoch 1/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 4.3333e-04 - val_loss: 2.9342e-06
Epoch 2/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6102e-05 - val_loss: 2.3481e-06
Epoch 3/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8.1325e-06 - val_loss: 2.3474e-06
Epoch 4/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3491e-06 - val_loss: 2.3492e-06
Epoch 5/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.0946e-06 - val_loss: 2.3439e-06
Epoch 6/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.4842e-06 - val_loss: 2.3427e-06
Epoch 7/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.0119e-06 - val_loss: 2.3402e-06
Epoch 8/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.0324e-06 - val_loss: 2.3382e-06
Epoch 9/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7070e-06 - val_loss: 2.3372e-06
Epoch 10/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.6771e-06 - val_loss: 2.3363e-06
Epoch 11/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/

In [66]:
def haversine_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters

def predict_over_distance(model, initial_point, max_distance=1000.0, time_step=60):
    predictions = []
    cumulative_distance = 0.0
    current_point = initial_point

    while cumulative_distance < max_distance:
        predicted_point = model.predict(x={
            'latitude': np.array([[current_point[1]]]),
            'longitude': np.array([[current_point[2]]]),
            'time': np.array([[current_point[0]]]),
        }, verbose=0)[0]
        print(predicted_point)

        # Calculate distance with the denormalized coordinates
        dist = haversine_distance(current_point[1], current_point[2], predicted_point[0], predicted_point[1])
        cumulative_distance += dist

        print(f"Current: ({current_point[1]:.6f}, {current_point[2]:.6f}) | "
              f"Predicted: ({predicted_point[0]:.6f}, {predicted_point[1]:.6f}) | "
              f"Distance : {dist:.2f} m")

        # Update time, keeping it in its original scale
        new_time = current_point[0] + time_step

        # Store prediction in original scale
        predictions.append([new_time, predicted_point[0], predicted_point[1]])

        # Update current point for next iteration, but time should remain in original scale
        current_point = [new_time, predicted_point[0], predicted_point[1]]

        # Break if cumulative distance meets or exceeds the target
        if cumulative_distance >= max_distance:
            break

    return predictions

initial_time = df['time'].iloc[-1]
initial_lat = df['lat'].iloc[-1]
initial_lon = df['lon'].iloc[-1]
initial_point = [initial_time, initial_lat, initial_lon]

predicted_path = predict_over_distance(model, initial_point)

# Convert predictions to a DataFrame for easy viewing
predicted_df = pd.DataFrame(predicted_path, columns=['time', 'lat', 'lon'])
print(predicted_df)

[ 20.993769 105.8682  ]
Current: (20.994061, 105.868448) | Predicted: (20.993769, 105.868202) | Distance : 41.23 m
[ 20.993769 105.8682  ]
Current: (20.993769, 105.868202) | Predicted: (20.993769, 105.868202) | Distance : 0.00 m
[ 20.993769 105.8682  ]
Current: (20.993769, 105.868202) | Predicted: (20.993769, 105.868202) | Distance : 0.00 m
[ 20.993769 105.8682  ]
Current: (20.993769, 105.868202) | Predicted: (20.993769, 105.868202) | Distance : 0.00 m
[ 20.993769 105.8682  ]
Current: (20.993769, 105.868202) | Predicted: (20.993769, 105.868202) | Distance : 0.00 m
[ 20.993769 105.8682  ]
Current: (20.993769, 105.868202) | Predicted: (20.993769, 105.868202) | Distance : 0.00 m
[ 20.993769 105.8682  ]
Current: (20.993769, 105.868202) | Predicted: (20.993769, 105.868202) | Distance : 0.00 m
[ 20.993769 105.8682  ]
Current: (20.993769, 105.868202) | Predicted: (20.993769, 105.868202) | Distance : 0.00 m
[ 20.993769 105.8682  ]
Current: (20.993769, 105.868202) | Predicted: (20.993769, 105.8

KeyboardInterrupt: 